# 端到端体验AIGC-从SageMaker到Web应用

生成式AI技术正在迅速提高，现在可以简单地根据文本输入来生成文本和图像。 本次动手训练营将采用Stable Diffusion🧨模型。

Stable Diffusion🧨是一种文本到图像模型，可让您创建逼真的AIGC应用程序。

在本次动手训练营中，您将了解到:
 1. Stable Diffusion模型生成图片的过程
 2. 在Sagemaker Notebook Instance中运行该模型
 3. 使用Sagemaker Notebook Instance部署模型并进行推理
 
---

## 1.准备工作

### 1.1 安装及环境配置工作 

#### 检查环境版本

In [ ]:
!nvcc --version
!pip list | grep torch

#### 安装Notebook运行模型所需的库文件

In [ ]:
!sudo yum-config-manager --disable docker-ce-stable
!sudo yum -y install pigz
!pip install -U pip
!pip install -U transformers==4.29.2 diffusers==0.16.1 ftfy accelerate
!pip install -U torch==1.13.1+cu117 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -U sagemaker
!pip list | grep torch

### 1.2 下载模型文件

#### 目前Stable Diffusion发布了Stable Diffusion V1和Stable Diffusion V2版本。

起初的[Stable Diffusion V1](https://github.com/CompVis/stable-diffusion) 由 [CompVis](https://ommer-lab.com) 领导，改变了开源人工智能模型的性质，并在全球范围内催生了数百个其他模型和创新。它是所有软件中最快达到 10K Github star 的软件之一，在不到两个月的时间内飙升至 33K star。

Stable Diffusion V1使用*降采样因子8*(downsampling-factor 8)自动编码器和860M UNet和 CLIP ViT-L/14文本编码器用于扩散模型。该模型在 256x256 图像上进行预训练，然后在 512x512 图像上进行微调。

与最初的 V1 版本相比，[Stable Diffusion V2](https://github.com/Stability-AI/StableDiffusion) 提供了许多重大改进和特性，具体表现在：

Stable Diffusion V2版本包含一个具有鲁棒性的文本生成图像模型，在全新的文本编码器 (OpenCLIP) 上训练而成，与早期的 V1 版本相比，文本 -> 图像模型大大提高了图像生成质量，可以生成默认分辨率为 512x512 像素和 768x768 像素的图像。

- [Stable Diffusion Launch Announcement](https://stability.ai/blog/stable-diffusion-announcement)
- [Stable Diffusion 2.0 Release](https://stability.ai/blog/stable-diffusion-v2-release)

#### Stable Diffusion的开源模型文件存放在Hugging Face上。

Hugging Face🤗 是一个 AI/ML 社区和平台，早期靠 Transformers 模型库和高质量社区受到关注。用户可以在 Hugging Face 上托管和共享 ML 模型、数据集，也可以构建、训练和部署模型。截至目前，Hugging Face 上共有 7.7 万个预训练模型，以 NLP 模型为主，目前 NLP 模型占比为 50%，2022 年初为 70%，这一比例未来会继续下降。Hugging Face 现在是 NLP 领域的 GitHub，未来希望成为整个 ML 领域的 GitHub，并逐渐向 ML Workflow 的其他环节渗透。

### 安装git lfs以克隆模型仓库 （git lfs支持大文件上传与下载）

In [ ]:
!sudo sudo amazon-linux-extras install epel -y
!sudo yum-config-manager --enable epel
!sudo yum install git-lfs -y

### 设定模型版本的环境变量

In [ ]:
# Clone the Stable Diffusion model from HuggingFace

#### Stable Diffusion V1
SD_SPACE="runwayml/"
SD_MODEL = "stable-diffusion-v1-5"
SD_EXCLUDE_MODEL="!v1-5-pruned.ckpt"

#### Stable Diffusion V2
# SD_SPACE="stabilityai/"
# SD_MODEL = "stable-diffusion-2-1"
# SD_EXCLUDE_MODEL="!v2-1_768-nonema-pruned.ckpt"

### 克隆模型仓库

In [ ]:
# Estimated time to spend v1(3min),v2(4min)
%cd ~/SageMaker
!echo $(date)
!printf "=======Current Path========%s\n"
!rm -rf $SD_MODEL
!mkdir $SD_MODEL
%cd $SD_MODEL
!git init
#Include / Exclude file
!git config core.sparseCheckout true
!echo "/*" >> .git/info/sparse-checkout
!echo "!**/*.safetensors" >> .git/info/sparse-checkout
!echo $SD_EXCLUDE_MODEL >> .git/info/sparse-checkout
#Checkout and pull file
!git remote add -f master https://huggingface.co/$SD_SPACE$SD_MODEL
!git pull master main
!rm -rf .git
%cd ~/SageMaker
!printf "=======Folder========%s\n$(ls)\n"
!echo $(date)

---
## 2.在Notebook中配置并使用模型

### 加载模型

In [ ]:
import torch
import datetime
from diffusers import StableDiffusionPipeline
# Load stable diffusion
pipe = StableDiffusionPipeline.from_pretrained(SD_MODEL, torch_dtype=torch.float16)

### 使用GPU进行运算并设定参数

为模型设定输入参数，可使用的部分参数如下：
- prompt (`str` or `List[str]`):
  - 引导图像生成的文本提示或文本列表
- height (`int`, *optional*, 默认为 V1模型可支持到512像素，V2模型可支持到768像素):
  - 生成图像的高度（以像素为单位）
- width (`int`, *optional*,  默认为 V1模型可支持到512像素，V2模型可支持到768像素):
  - 生成图像的宽度（以像素为单位）
- num_inference_steps (`int`, *optional*, defaults to 50):
  - 降噪步数。更多的去噪步骤通常会以较慢的推理为代价获得更高质量的图像
- guidance_scale (`float`, *optional*, defaults to 7.5):
  - 较高的指导比例会导致图像与提示密切相关，但会牺牲图像质量。 如果指定，它必须是一个浮点数。 guidance_scale<=1 被忽略。
- negative_prompt (`str` or `List[str]`, *optional*):
  - 不引导图像生成的文本或文本列表。不使用时忽略，必须与prompt类型一致（如果 guidance_scale 小于 1 则忽略）
- num_images_per_prompt (`int`, *optional*, defaults to 1):
  - 每个提示生成的图像数量
  
更多参数请参考：https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py



*GPU内存不够怎么办？*
- *试一试分辨率小一点的图片*
- *减少生成图片的数量*
- *升级机型*

In [ ]:
# move Model to the GPU
torch.cuda.empty_cache()
pipe = pipe.to("cuda")

# V1 Max-H:512,Max-W:512
# V2 Max-H:768,Max-W:768

print(datetime.datetime.now())
prompts =[
    "Eiffel tower landing on the Mars",
    "a photograph of an astronaut riding a horse,van Gogh style",
]
generated_images = pipe(
    prompt=prompts,
    height=512,
    width=512,
    num_images_per_prompt=1
).images  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)

print(f"Prompts: {prompts}\n")
print(datetime.datetime.now())

for image in generated_images:
    display(image)


---
## 3.部署模型至Sagemaker Inference Endpoint

构建和训练模型后，您可以将模型部署至终端节点，以中获取预测推理结果。

使用 SageMaker 托管服务部署模型有多种选择。 你可以使用 AWS 开发工具包（例如，Python 开发工具包 (Boto3)）、SageMaker Python 开发工具包、AWS CLI 以编程方式部署模型， 或者您可以使用 SageMaker 控制台以交互方式部署模型。

使用 SageMaker 托管服务部署模型是一个三步过程，如果您使用 适用于 Python (Boto3)、AWS CLI 或 SageMaker 控制台的 AWS 开发工具包：
    
    1.在 SageMaker 中创建 SageMaker 模型。
    2.为 HTTPS 端点创建端点配置。
    3.创建 HTTPS 端点。
    
使用 SageMaker Python 开发工具包部署模型不需要您创建终端节点配置。 因此，这是一个两步过程：
    
    1.从创建模型对象 Model可以部署到 HTTPS 端点的类。
    2.使用模型对象的预构建创建 HTTPS 端点 deploy()方法。 

### 编写初始化的Sagemaker代码用于部署推理终端节点

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None

if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

### 创建自定义推理 inference.py 脚本

要使用自定义推理脚本，您需要创建一个 inference.py 脚本。 
在我们的示例中，我们将编写 model_fn 以正确加载我们的模型，并编写 predict_fn 以处理数据。

更多方法，请参考部署HuggingFace模型到Sagemaker中：https://huggingface.co/docs/sagemaker/inference

In [ ]:
!mkdir ./$SD_MODEL/code

#### 为模型创建所需依赖声明的文件

In [ ]:
%%writefile ./$SD_MODEL/code/requirements.txt
diffusers==0.16.1
transformers==4.29.2

In [ ]:
%%writefile ./$SD_MODEL/code/inference.py
import base64
import torch
from io import BytesIO
from diffusers import StableDiffusionPipeline


def model_fn(model_dir):
    # Load stable diffusion and move it to the GPU
    pipe = StableDiffusionPipeline.from_pretrained(model_dir, torch_dtype=torch.float16)
    pipe = pipe.to("cuda")

    return pipe


def predict_fn(data, pipe):

    # get prompt & parameters
    prompt = data.pop("prompt", "")
    # set valid HP for stable diffusion
    height = data.pop("height", 512)
    width = data.pop("width", 512)
    num_inference_steps = data.pop("num_inference_steps", 50)
    guidance_scale = data.pop("guidance_scale", 7.5)
    num_images_per_prompt = data.pop("num_images_per_prompt", 1)
    # run generation with parameters
    generated_images = pipe(
        prompt=prompt,
        height=height,
        width=width,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        num_images_per_prompt=num_images_per_prompt,
    )["images"]

    # create response
    encoded_images = []
    for image in generated_images:
        buffered = BytesIO()
        image.save(buffered, format="JPEG")
        encoded_images.append(base64.b64encode(buffered.getvalue()).decode())

    # create response
    return {"generated_images": encoded_images}

#### 打包模型并上传至S3桶

In [ ]:
#Package model, Estimated time to spend 5min
!echo $(date)

!tar --exclude .git --use-compress-program=pigz -pcvf ./$SD_MODEL'.tar.gz' -C ./$SD_MODEL/ .

!echo $(date)

In [ ]:
from sagemaker.s3 import S3Uploader


print(datetime.datetime.now())
# upload model.tar.gz to s3, Estimated time to spend 30s(V1), 1min(V2)
sd_model_uri=S3Uploader.upload(local_path=f"{SD_MODEL}.tar.gz", desired_s3_uri=f"s3://{sess.default_bucket()}/stable-diffusion")
print(f"=======S3 File Location========\nmodel uploaded to:\n{sd_model_uri}")

print(datetime.datetime.now())

#### 使用HuggingFace将模型部署至SageMaker

In [ ]:
#init variables
huggingface_model = {}
predictor = {}

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel
# create Hugging Face Model Class
huggingface_model[SD_MODEL] = HuggingFaceModel(
    model_data=sd_model_uri, # path to your model and script
    role=role, # iam role with permissions to create an Endpoint
    transformers_version="4.26.0", # transformers version used
    pytorch_version="1.13.1", # pytorch version used
    py_version='py39', # python version used
)

In [ ]:
# deploy the endpoint endpoint, Estimated time to spend 5min(V1), 8min(V2)
print(datetime.datetime.now())

predictor[SD_MODEL] = huggingface_model[SD_MODEL].deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",
    endpoint_name=f"{SD_MODEL}-endpoint"
)

print(f"\n{datetime.datetime.now()}")
print(f"\n{SD_MODEL}-endpoint")

#### 基于推理终端节点生成自定义图片

In [ ]:
from PIL import Image
from io import BytesIO
import base64

# helper decoder
def decode_base64_image(image_string):
    base64_image = base64.b64decode(image_string)
    buffer = BytesIO(base64_image)
    return Image.open(buffer)


In [ ]:
#run prediction
response = predictor[SD_MODEL].predict(data={
    "prompt": [
        "Eiffel tower landing on the Mars",
#         "a photograph of an astronaut riding a horse",
    ],
    "height" : 512,
    "width" : 512,
    "num_images_per_prompt":1
  }
)

#decode images
decoded_images = [decode_base64_image(image) for image in response["generated_images"]]

#visualize generation
for image in decoded_images:
    display(image)
